In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import sys
import math

sys.path.insert(0, '/home/bay001/projects/codebase/rbp-maps/maps/')
from pandas.util.testing import assert_frame_equal
from density import ReadDensity
from density import normalization_functions as norm
from density import deprecated_interval_functions
from density import intervals
from density import Feature

import math

pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', 1500)

wd = '/home/bay001/projects/codebase/rbfox2/'

regions = os.path.join(
    wd,
    'RBFOX2-BGHLV26-HepG2-SE.MATS.JunctionCountOnly.txt'
)

ip_bam = os.path.join(
    wd,
    '204_01_RBFOX2.merged.r2.bam'
)
input_bam = os.path.join(
    wd,
    'RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam'
)

ip_pos_bw = os.path.join(
    wd,
    '204_01_RBFOX2.merged.r2.norm.neg.bw'
) # flipped?
ip_neg_bw = os.path.join(
    wd,
    '204_01_RBFOX2.merged.r2.norm.pos.bw'
) # flipped?
input_pos_bw = os.path.join(
    wd,
    'RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.norm.neg.bw'
) # flipped?
input_neg_bw = os.path.join(
    wd,
    'RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.norm.pos.bw'
) # flipped?

ip_read_density = ReadDensity.ReadDensity(
    pos = ip_pos_bw,
    neg = ip_neg_bw,
    bam = ip_bam
)

input_read_density = ReadDensity.ReadDensity(
    pos = input_pos_bw,
    neg = input_neg_bw,
    bam = input_bam
)
annotation_file = '/home/bay001/projects/codebase/rbfox2/RBFOX2-BGHLV26-HepG2-SE.MATS.JunctionCountOnly.txt'

def nan_to_num(somelist,num=-1):
    return [num if math.isnan(x) else x for x in somelist]

/home/bay001/projects/codebase/rbfox2/204_01_RBFOX2.merged.r2.bam
/home/bay001/projects/codebase/rbfox2/RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam


In [2]:
exon_offset = 50
intron_offset = 300
with open(annotation_file,'r') as f:
    f.next()
    for line in f:
        f = Feature.SkippedExonFeature(
            annotation=line.strip(),
            source='rmats'
        )
        up, skip, down = f.get_bedtools()
        ofpad,original,otpad = deprecated_interval_functions.five_prime_site(ip_read_density,skip,down,exon_offset,intron_offset)
        bfpad,better,btpad = intervals.five_prime_site(ip_read_density,skip,down,exon_offset,intron_offset)
        try:
            assert(ofpad == bfpad)
            assert(otpad == btpad)
            assert(
                nan_to_num(original)==nan_to_num(better)
            )
        except AssertionError:
            print(ofpad,bfpad,up,skip, otpad,btpad, len(original), len(better), skip.strand)
            break

In [3]:
exon_offset = 50
intron_offset = 300
with open(annotation_file,'r') as f:
    f.next()
    for line in f:
        f = Feature.SkippedExonFeature(
            annotation=line.strip(),
            source='rmats'
        )
        up, skip, down = f.get_bedtools()
        ofpad,original,otpad = deprecated_interval_functions.three_prime_site(ip_read_density,up,skip,exon_offset,intron_offset)
        bfpad,better,btpad = intervals.three_prime_site(ip_read_density,up,skip,exon_offset,intron_offset)
        try:
            assert(ofpad == bfpad)
            assert(otpad == btpad)
            assert(
                nan_to_num(original)==nan_to_num(better)
            )
        except AssertionError:
            print(ofpad,bfpad,up,skip, otpad,btpad, len(original), len(better), skip.strand)
            break

# Test without neighboring exon

In [4]:
exon_offset = 50
intron_offset = 300
with open(annotation_file,'r') as f:
    f.next()
    for line in f:
        f = Feature.SkippedExonFeature(
            annotation=line.strip(),
            source='rmats'
        )
        up, skip, down = f.get_bedtools()
        bfpad,better,btpad = intervals.five_prime_site(ip_read_density,None,skip,exon_offset,intron_offset)
        if(bfpad+len(better)+btpad) != (exon_offset + intron_offset):
            print("BAD!")